In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from config import akey

In [2]:
file = 'api-data/stock_prices.csv'
stock_prices = pd.read_csv(file)
company_file = 'api-data/company_info.csv'
company = pd.read_csv(company_file)
#Setting date column to be date/time object for time series analysis
stock_prices['index.1'] = pd.to_datetime(stock_prices['index.1'], format='%Y-%m-%d') 
#Renaming columns 
stock_prices.rename(columns={'index':'old_index', 'index.1': 'Date', '1. open': 'Open', '2. high':'High', 
                             '3. low':'Low','4. close':'Close', '5. volume':'Daily Volume'}, inplace=True)
#dropping un-needed columns in existing dataframe
stock_prices.drop(columns=['old_index'], inplace=True)
#setting index to Date for analysis
stock_prices.set_index('Date', inplace=True)
stock_prices['Year'] = pd.DatetimeIndex(stock_prices.index).year
stock_prices['Month'] = pd.DatetimeIndex(stock_prices.index).month
stock_prices.to_csv('api-data/cleaned_stock_prices.csv')
stock_prices.reset_index(inplace=True)
stock_prices.head()

,Date,Open,High,Low,Close,Daily Volume,Ticker,Year,Month
0,2020-12-15,21.70,22.11,16.580,17.99,23931715.0,AAN,2020,12
1,2020-12-15,21.70,22.11,16.580,17.99,23931715.0,AAN,2020,12
2,2020-12-15,8.33,8.53,7.620,8.41,6979264.0,AAOI,2020,12
3,2020-11-30,8.82,10.20,7.200,8.31,17670623.0,AAOI,2020,11
4,2020-10-30,11.40,11.94,8.469,8.78,8911887.0,AAOI,2020,10


In [3]:
company.head()
#stock_prices.head()

,Ticker,Sector,Industry,Full Time Emp
0,AAOI,Technology,Semiconductors,3115
1,AAON,Industrials,Building Products & Equipment,2290
2,AAT,Real Estate,REIT-Diversified,199
3,AAWW,Industrials,Airports & Air Services,3587
4,ABCB,Financial Services,Banks-Regional,2656


In [4]:
#adding company information to stock prices file
all_info= stock_prices.merge(company, on='Ticker', how='left')
all_info.set_index('Date', inplace=True)
all_info.sort_index(ascending=True, inplace=True)
all_info['Year'] = pd.DatetimeIndex(all_info.index).year
all_info['Month'] = pd.DatetimeIndex(all_info.index).month
#drop remaining unneeded columns
all_info.drop(columns=['Open','High','Low','Daily Volume'], inplace=True)
all_info.to_csv('api-data/all_info.csv')
all_info.head()

,Close,Ticker,Year,Month,Sector,Industry,Full Time Emp
Date,,,,,,,
1999-12-31,20.00,CLB,1999,12,Energy,Oil & Gas Equipment & Services,3800.0
1999-12-31,10.38,CENT,1999,12,Consumer Defensive,Packaged Foods,5600.0
1999-12-31,0.08,RDNT,1999,12,Healthcare,Diagnostics & Research,6165.0
1999-12-31,11.63,ESE,1999,12,Technology,Scientific & Technical Instruments,2713.0
1999-12-31,20.75,FBP,1999,12,Financial Services,Banks-Regional,2674.0


In [5]:
#This block loops through the date indices and creates separate files for each year for all stock 
#closing prices and outputs to CSV
years = ['2015','2016','2017','2018','2019','2020']
for year in years:
    year = all_info[f'{year}-01-01':f'{year}-12-31'].to_csv(f'api-data/yearly_summary/prices_for_{year}.csv')

In [6]:
#analyzing 2020 data to find interesting sectors
file='api-data/yearly_summary/prices_for_2020.csv'
stocks2020 = pd.read_csv(file)
#lamba function takes each stock ticker in the CSV, finds the first date and the last date and then calculates % chg
sect_perf = stocks2020.groupby(['Ticker','Sector','Industry'],sort=False).apply(lambda x: (x['Close'].values[-1] - x['Close'].values[0]) / x['Close'].values[-1] * 100)\
    .reset_index(name='pct change')
sect_perf.head()

,Ticker,Sector,Industry,pct change
0,TRST,Financial Services,Banks-Regional,-21.068702
1,LNTH,Healthcare,Diagnostics & Research,-22.791024
2,EPRT,Real Estate,REIT-Diversified,-30.174446
3,MDC,Consumer Cyclical,Residential Construction,14.384397
4,VRA,Consumer Cyclical,Footwear & Accessories,-22.978177


In [7]:
sector = sect_perf.groupby('Sector').mean()
sector.sort_values('pct change', ascending=False, inplace=True)
sector.to_csv('api-data/sector_chgs.csv')
sector.head(25)

,pct change
Sector,
Technology,13.548758
Basic Materials,12.188316
Consumer Defensive,9.328190
Industrials,7.555920
Healthcare,7.233188
Consumer Cyclical,6.285675
Financial Services,-7.277631
Communication Services,-9.482677
Utilities,-16.221258


In [8]:
sector.describe()

,pct change
count,11.000000
mean,-2.834739
std,15.628285
min,-33.506755
25%,-12.851968
50%,6.285675
75%,8.442055
max,13.548758
